# **Power System Simulation Final Presentation FUSE**

## **Introduction**
Welcome to the final presentation on Power System Simulation. In this presentation, we will cover the following packages:

`graph_processing.py`
- Initialization and validation
- Find downstream vertices
- Find alternatie edges

`power_grid_modelling.py`
- Handle PGM input format
- Aggregate power flow results in tables
- Show tables


`grid_analytic.py`
- Calculate EV penetration levels
- Calculate optimal tap positions
- Do N-1 calculations

We have tested all the packages using `pytest` to ensure there are no errors and that the output is as expected. For performance checking, we will also run a timer to check how fast the package would run.

To be able to use the package, users must install the following dependencies (libraries):
* `numpy`
* `pandas`
* `networkx`
* `power-grid-model`
* `math`
* `random`

In [1]:
# Create a timer class
import time

class Timer:
    def __init__(self, name: str):
        self.name = name
        self.start = None

    def __enter__(self):
        self.start = time.perf_counter()

    def __exit__(self, *args):
        print(
            f'Execution time for {self.name} is {(time.perf_counter() - self.start):0.6f} s'
        )

In [8]:
# Importing all functions from the package
from power_system_simulation.graph_processing import GraphProcessor
from power_system_simulation.power_grid_modelling import PowerGridModelling
from power_system_simulation.grid_analytic import GridAnalysis

## **1.Graph-Processing**

This section covers the functionalities and usage of the `graph_processing.py` package. The `graph_processing.py` package builds an undirected graph and implements two functionalties namely, `find_downstream_vertices()` and `find_alternative_edges()`.
This section covers the functionalities and usage of the `graph_processing.py` package. The `graph_processing.py` package builds an undirected graph and implements two functionalties namely, `find_downstream_vertices()` and `find_alternative_edges()`.

### **1.1 Input data**

To create a graph the following parameters need to be defined:
* `edge_ids`
* `edge_vertex_id_pairs`
* `edge_enabled`
* `source_vertex_id`

For example, we have a network created as follow:

      ------- edge_1---- source_0 ------ edge_5 -----
      |                    |                       |
      |                  edge_3                    |
      |                    |                       |
    vertex_2 -- edge_7 -- vertex_4  -- edge_8 --vertex_6
      |
    edge_9
      |
    vertex_10  

If we only take into account the enabled edges:

      ------- edge_1---- source_0 ------ edge_5 -----
      |                    |                       |
      |                  edge_3                    |
      |                    |                       |
    vertex_2            vertex_4                vertex_6
      |
    edge_9
      |
    vertex_10  

In [13]:
# Create input data
# List of edge IDs.    
edge_ids = [1, 3, 5, 7, 8, 9]
# Tuples representing connections between vertices, defining the graph's edges.
edge_vertex_id = [(0, 2), (0, 4), (0, 6), (2, 4), (4, 6), (2, 10)]
# List indicating whether each edge is enabled or disabled.
edge_enabled = [True, True, True, False, False, True]
# ID of the vertex that is also a source.
source_id = 0

# Calling the class
result = GraphProcessor(
    edge_ids=edge_ids,
    edge_vertex_id_pairs=edge_vertex_id,
    edge_enabled=edge_enabled,
    source_vertex_id=source_id,
)

If we want to know what is the downstream vertices at a given edge (for example edge 1) with respect to the source:

In [14]:
edge_id = 1

downstream_vertices = result.find_downstream_vertices(edge_id=edge_id)
print(downstream_vertices)

[2, 10]


And if we want to know the alternative adges to be connected if we disconnect the edge 5 

In [17]:
edge_id = 5

alternative_eges = result.find_alternative_edges(edge_id)
print(alternative_eges)

[8]


## **2 Power Grid Modelling Package**
In this assignment, we used the `power_grid_model` package as our core package for calculations.

We also used `pandas` and `numpy` for dataframes and ndarrays related functionality. For coherence with previous assignment, we also created a class called PowerGridModelling, along with some expected errors.
We assumed that the input data and the load profiles will be given in standard PGM format, therefore we expected users to have the input data as json file and load profile files as parquet files. From that, users can conveniently paste the paths of the file into our function.

In [4]:
import numpy as np
import pandas as pd
import json
import scipy

from power_grid_model import (
    LoadGenType,
    PowerGridModel,
    CalculationType,
    CalculationMethod,
    initialize_array
)

from power_grid_model.validation import (
    assert_valid_input_data,
    assert_valid_batch_data
)

from power_grid_model.utils import (
    json_deserialize
)

### **2.1 Input Data**
* A power grid in PGM input format
* A table containing active load profile of all the (`sym_load`) in the grid, with timestamps and load ids.
* A table containing reactive load profile of all the (`sym_load`) in the grid, with timestamps and load ids.




In [6]:
# Define file paths
data_path = "data/input_network_data.json"
active_load_profile_path = "data/active_power_profile.parquet"
reactive_load_profile_path = "data/reactive_power_profile.parquet"

# Load JSON data
with open(data_path) as fp:
    dataset = json.load(fp)

# Load Parquet data
active_load_profile = pd.read_parquet(active_load_profile_path)
reactive_load_profile = pd.read_parquet(reactive_load_profile_path)

The `dataset` encompasses multiple tables that detail various components of the electrical grid, including nodes, lines, sources, and load details.

In [ ]:
node_data = dataset['data']['node']
df_node_data = pd.DataFrame(node_data)
display(df_node_data)

line_data = dataset['data']['line']
df_line_data = pd.DataFrame(line_data)
display(df_line_data)

sym_load_data = dataset['data']['sym_load']
df_sym_load_data = pd.DataFrame(sym_load_data)
display(df_sym_load_data)

source_data = dataset['data']['source']
df_source_data = pd.DataFrame(source_data)
display(df_source_data)

The `active_power_profile` and `reactive_power_profile` provide detailed timelines of power requirements for various loads, indicating the amount of power that needs to be supplied.

In [ ]:
display(active_load_profile)
display(reactive_load_profile)

### **2.2 Power Flow Results in Tables**

A table with each row representing a timestamp, with the following columns:
- Timestamp (index column)
- Maximum p.u. voltage of all the nodes for this timestamp
- The node ID with the maximum p.u. voltage
- Minimum p.u. voltage of all the nodes for this timestamp
- The node ID with the minimum p.u. voltage

A table with each row representing a line, with the following columns:
- Line ID (index column)
- Energy loss of the line across the timeline in kWh
- Maximum loading in p.u. of the line across the whole timeline
- Timestamp of this maximum loading moment
- Minimum loading in p.u. of the line across the whole timeline
- Timestamp of this minimum loading moment

In [ ]:
# Initialize the PowerGridModelling class
pgm = PowerGridModelling(data_path, active_load_profile_path, reactive_load_profile_path)

df_per_timestamp = pgm.data_per_timestamp()
df_per_line = pgm.data_per_line()

# Display the results
print("\nData Per Timestamp:")
display(df_per_timestamp.head())

print("\nData Per Line:")
display(df_per_line.head())
# display(DataFrame(df_per_line.head()))

## **3. Grid Analysis**
In this part a package with some low voltage (LV) functions is presented. A LV grid with one transformer, one source and many `sym_load` representing houses is condiderd in this part.

### **3.1 input data**
The class called `GridAnalysis` is used for this package, it takes these arguments:
* Path of LV grid network data (`data_path`)
* List of LV feeder IDs (`feeder_ids`)
* Path of active load profile (`active_load_profile_path`)
* Path of reactive load profile (`reactive_load_profile_path`)
* Path of pool of EV charging profiles (`ev_pool_path`)

The input data is loaded as follows:

In [11]:
data_path="data/big_network/input_network_data.json"
feeder_ids=[1204,1304,1404,1504,1604,1704,1804,1904]
active_load_profile_path="data/big_network/active_power_profile.parquet"
reactive_load_profile_path="data/big_network/reactive_power_profile.parquet"
ev_pool_path="data/big_network/ev_active_power_profile.parquet"

data = [data_path, active_load_profile_path, reactive_load_profile_path, ev_pool_path]

data = GridAnalysis(data, feeder_ids)

[803, 1204, 1205, 1206, 1207, 1208, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369

The input data is checked for the following validity criteria:

1. checks done in this part:
  * The LV grid has exactly one transformer, and one source.
  * All IDs in the LV Feeder IDs are valid line IDs.
  * All the lines in the LV Feeder IDs have the from_node the same as the to_node of the transformer.
  * The timestamps are matching between the active load profile, reactive load profile, and EV charging profile.
  * The number of EV charging profile is at least the same as the number of sym_load.

2. checks done in first part:
  * The grid is fully connected in the initial state.
  * The grid has no cycles in the initial state.

3. checks done in second part:
  * The LV grid should be a valid PGM input data.
  * The IDs in active load profile and reactive load profile are matching.
  * The IDs in active load profile and reactive load profile are valid IDs of sym_load.

### **3.2 EV penetration level**

The EV penetration level function adds EV charging profiles to a percentage of houses which has EV charged at home (EV penetration level). The houses and charging profiles are chosen randomly. This is done as follows:

1. The number of EVs per LV feeder are calculated using: `[penetration_level * total_houses / number_of_feeders]`
2. The houses attached to each LV feeder are indentified using the function `find_downstream_vertices` from part 1
3. For each LV feeder a random charging profile is added to the specified number of random houses
4. The function `powerGridModeling` from part 2 is than called to get the new results

In [ ]:
penetration_level = 0.2

with Timer("EV penetration level"):
  df_result_per_timestamp, df_result_per_line = data.EV_penetration_level(penetration_level)

print("u_pu per timestamp results:")
print(df_result_per_timestamp)
print("loading per line results:")
print(df_result_per_line)

### **3.3 N-1 calculation**

In [ ]:
edge_id = 22

with Timer("N-1 calculation"):
  result = data.alternative_grid_topology(edge_id=edge_id)

print(result)
# **1.Graph-Processing**
